# Scraping journalists then using twint to get who they follow

In [ ]:
import sys
import os
import time
import json
import csv
import threading
import queue
import asyncio 
import nest_asyncio
nest_asyncio.apply()
import twint

sys.path.insert(1, 'C:/Users/Luca/Aug20_Ditchley')
from src.data import journalists as journos

### 1. Getting journalist twitter handles according to topic

In [3]:
keyword = 'politics'
journo_handles = journos.get_handles_by_keyword(keyword)
print(len(journo_handles))

239


In [ ]:
print(journo_handles)

In [ ]:
#this journalist has a locked account
journo_handles.remove('P_Madeley_Star')
#journo_handles.remove('AndyRoocroft')

## 2. Loop over journalists and get who they follow

### 2.2 Storing each threads results as csv files (USE THIS)

This seems to work quite nicely, and may even be more reliable than storing in memory.  

Note that any recursive use to get data that was missed has to be done manually by alternately using the last two cells before section 3 to recheck the failed list and run it.

In [ ]:
num_threads = 6 # num_threads can be any amount, but only so many can be run in parallel, according to number of cores you have.
# import multiprocessing; multiprocessing.cpu_count() can tell you how many cores python can see.

In [ ]:
def get_friends(q, fp, persist):
    '''
    Params
    ------
    q : a Queue instance
    
    fp : string
        Provides the path where the file should be saved.
    persist : bool
        If True, make a repeat attempt if attempt fails.
    '''
    while True: 
        username = q.get()
        success = False
        filepath = fp+'friends_'+username+'.csv'
        while not success:
            print('Attempting to get friends of @'+username)
            c = twint.Config()
            c.Username = username
            c.User_full = False
            c.Hide_output = True
            c.Output = filepath

            twint.run.Following(c)
            
            if persist:
                if os.path.exists(filepath):
                    success = True
                    print('Friends of @'+username+' saved.')
            else:
                success = True
        q.task_done()

In [ ]:
q = queue.Queue(maxsize=0)

for i in range(num_threads): # Loop to create threads
    worker = threading.Thread(target=get_friends, args=(q, '../data/raw/'+keyword+'_', False)) # Change persist to True to tell the code to keep trying until results are written to file.
    worker.setDaemon(True)
    worker.start()

# for username in journo_handles: # Loop to add journalists usernames to the queue

    q.put(username)

q.join()

In [ ]:
all_handles = []
all_users = []
failed = []
for name in journo_handles:
    filepath = '../data/raw/'+keyword+'_friends_'+name+'.csv'
    if not os.path.exists(filepath):
        failed.append(name)
    else:
        with open(filepath, newline='') as f:
            reader = csv.reader(f)
            handles = list(reader)
            all_handles.extend([handle[0] for handle in handles])
            all_users.extend([name for handle in handles])
            print('@'+name+' follows '+str(len(handles))+' users.')
            
print('\nTotal number of handles pulled: '+str(len(all_handles)))

unique = len(set(all_handles))
print('Number of unique twitter handles: '+str(unique))

print('\nZero following in list for users: '+str(failed))

If some of the requests seem to have failed, can try them again:

In [ ]:
q = queue.Queue(maxsize=0)

for i in range(num_threads): # Loop to create threads
    worker = threading.Thread(target=get_friends, args=(q, '../data/raw/'+keyword+'_', False))
    worker.setDaemon(True)
    worker.start()

for username in failed: # Loop to add journalists usernames to the queue
    q.put(username)

q.join()

Interestingly, most of the handles must appear only once, suggesting there are only a few users that are followed by more than one of the journalists in this list.

## 3. Store the data in one csv file using pandas

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(list(zip(all_users, all_handles)), 
               columns =['screen_name', 'friend'])

In [ ]:
df.head()

In [ ]:
df.to_csv('../data/raw/'+keyword+'_journalist_friends.csv', index=False)

## 4. Quick inspection of data

In [ ]:
from collections import Counter

In [ ]:
counted = Counter(all_handles)

In [ ]:
counted.most_common(50)

In [3]:
num_threads = 7 # num_threads can be any amount, but only so many can be run in parallel, according to number of cores you have.
# import multiprocessing; multiprocessing.cpu_count() can tell you how many cores python can see.

In [4]:
### from here I try the same but for tweets

def get_tweets(q, fp, persist):
    '''
    Params
    ------
    q : a Queue instance
    
    fp : string
        Provides the path where the file should be saved.
    persist : bool
        If True, make a repeat attempt if attempt fails.
    '''
    while True: 
        username = q.get()
        success = False
        filepath = fp +username+'.csv'
        while not success:
            print('Attempting to get tweets of @'+username)
            c = twint.Config()
            c.Username = username
            #c.User_full = False
            c.Hide_output = True
            #c.Pandas =True
            #c.Store_object = True
            c.Limit = 10000000
            c.Until = '2020-08-01'
            c.Since = '2019-08-01'
            c.Profile_full = True
            c.Store_csv = True
            c.Retweets = True
            #c.Native_retweets = True
            c.Output = filepath

            twint.run.Search(c) # Profile
            tweets = twint.storage.panda.Tweets_df
            #tweets = tweets.append(twint.storage.panda.Tweets_df)
            
            if persist:
                if os.path.exists(filepath):
                    success = True
                    print('Tweets of @'+username+' saved.')
            else:
                success = True
        q.task_done()

In [ ]:
q = queue.Queue(maxsize=0)

for i in range(num_threads): # Loop to create threads
    worker = threading.Thread(target=get_tweets, args=(q,'../data/raw/'+keyword +'_', True)) # Change persist to True to tell the code to keep trying until results are written to file.
    worker.setDaemon(True)
    worker.start()

for username in journo_handles: # Loop to add journalists usernames to the queue
#for username in ['jennystrasburg',]: 
    q.put(username)

q.join()

Attempting to get tweets of @paulcockerton
Attempting to get tweets of @DelModyAttempting to get tweets of @patrickwintour
Attempting to get tweets of @hugh_muir
Attempting to get tweets of @JohnDomokosAttempting to get tweets of @Davidmkeys


Attempting to get tweets of @elashton


CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable


Tweets of @hugh_muir saved.
Attempting to get tweets of @patrick_kidd


CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable


Tweets of @Davidmkeys saved.
Attempting to get tweets of @TimRoss_1


...

CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable


Attempting to get tweets of @dzuidijk
Attempting to get tweets of @LauraHusbo


CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 1.0 secs
CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable


Attempting to get tweets of @AndyRoocroft


CRITICAL:root:twint.get:User:'NoneType' object is not subscriptable
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 1.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 1.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 1.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 8.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 8.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 8.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 8.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 27.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping

In [5]:
all_handles = []
all_users = []
failed = []
for name in journo_handles:
    filepath = '../data/raw/'+keyword +'_'+name+'.csv'
    if not os.path.exists(filepath):
        failed.append(name)
    if os.path.exists(filepath):
        all_handles.append(filepath)        

In [6]:
failed

['Tinglepolitics', 'LauraHusbo', 'dzuidijk', 'AndyRoocroft']

In [ ]:
#this journalist has a locked account
failed.remove('Tinglepolitics')# no tweets in the target date period
#failed.remove('P_Madeley_Star') # locked tweets
failed.remove('LauraHusbo') # no tweets in the target date period
failed.remove('AndyRoocroft') # locked tweets
failed.remove('dzuidijk') # no tweets in the target date period
#failed.remove('journomatei') # no tweets in the target date period
#failed.append('mateirosca')
#journo_handles.remove('AndyRoocroft')

In [ ]:
q = queue.Queue(maxsize=0)

for i in range(num_threads): # Loop to create threads
    worker = threading.Thread(target=get_tweets, args=(q, '../data/raw/'+keyword+'_', True))
    worker.setDaemon(True)
    worker.start()

for username in failed: # Loop to add journalists usernames to the queue
    q.put(username)

q.join()

In [7]:
import pandas as pd
all_handles = []
all_users = []
failed = []
all_tweets = pd.DataFrame()
for name in journo_handles:
    filepath = '../data/raw/'+keyword +'_'+name+'.csv'
    if os.path.exists(filepath):
        all_handles.append(filepath)
        temp_csv = pd.read_csv(filepath)
        all_tweets = pd.concat([all_tweets, temp_csv])

In [8]:
len(all_tweets)

234365

In [ ]:
all_tweets.index = range(len(all_tweets))


In [9]:
all_tweets.to_csv('twint_politics_18082020_inParallel_with_index.csv', index = True)